In [ ]:
from pathlib import Path

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

from helpers import *
from derive_model import *

%load_ext autoreload
%autoreload 2

In [ ]:
# Parsing all power data for a given device
# .. assumed ran from the `automation` directory

# Select a device to analyse
device_id = 'ciscoNexus9336-FX2'
# device_id = 'ciscoNCS55A1-24H'
# device_id = 'N540X-8Z16G-SYS-A'
# device_id = 'cisco8201-32FH'
# device_id = 'aristaDCS-7280CR3K-32D4'
# device_id = 'aristaDCS-7280CR3K-32D4'

# Set paths
input_data_path = Path('..','data','router',device_id,'static')
output_data_path = Path('..','devices',device_id)

reprocess_power_logs = False
# reprocess_power_logs = True


In [ ]:
if reprocess_power_logs:

    # ==============
    # Process the power logs
    # ==============

    # Initialize the output dictionary
    power_data = {'device' : device_id}

    # Iterate through all the experiment types
    for dir in sorted(input_data_path.glob('*')):
        
        # Iterate through each run of an experiment
        for ts in sorted(dir.glob('*')):

            # Filter out the experiments done with only one PSU plugged in
            # date = ts.name.split('_')[0]
            # if date == '2024-10-18' or date == '2024-10-19':
            #     continue
            # .. S measurements were average power value is less than 1W
            #    on either channel
            tmp = pd.read_csv(ts/'power.log')
            if (float(tmp.iloc[:,0].mean()) < 1000 or 
                float(tmp.iloc[:,1].mean()) < 1000):
                print('Appears to have only one channel measuring.\nDiscarding {}'.format(ts))
                continue

            # Load the `metadata.yml` file
            # .. Fix files where the writing was broken
            try:
                metadata = load_yml(ts/'metadata-fixed.yml') 
            except FileNotFoundError:
                try: 
                    metadata = load_yml(ts/'metadata.yml')
                except:
                    # we have a buggy port_list
                    # -> clean and reload
                    clean_buggy_port_lists(ts/'metadata.yml')
                    metadata = load_yml(ts/'metadata-fixed.yml')

            # .. Patch for old data naming
            old_speed = (type(metadata['port_speed']) != str)
            if old_speed:
                if metadata['port_speed'] < 1000:
                    metadata['port_speed'] = str(metadata['port_speed'])+"M"
                    old_speed = False
            # .. Extract useful info from 
            group = {
                'port_type'     : metadata['port_type'],
                'trx'           : metadata['transceivers'],
                'exp_type'      : metadata['experiment_type'],
                'port_speed'    : int(metadata['port_speed']/1000) if old_speed else metadata['port_speed'],
                'n_ports'       : metadata['port_number'],
            }
            # .. correct for buggy port_types
            if group['port_type'] == 'PCC':
                group['port_type'] = 'QSFP'
            elif group['port_type'] == 'LR_SFP_10G':
                group['port_type'] = 'SFP+'
            elif group['port_type'] == 'LR_SFP_1G':
                group['port_type'] = 'SFP'

            # .. for the dynamic case, add the traffic-related metadata
            if 'packet_size_bytes' in metadata:
                group['packet_size_bytes']  = metadata['packet_size_bytes']
                # .. sensibly round the value of achieved bandwidth
                exact_bw = metadata['bandwidth_reached_gbps']
                # Adjust for the low bandwidth cases.
                if metadata['bandwidth_gbps'] in [1]:
                    rounded_bw = round(exact_bw,1)
                elif metadata['bandwidth_gbps'] in [0.1,0.5]:
                    rounded_bw = round(exact_bw,2)
                elif metadata['bandwidth_gbps'] in [0.01,0.05]:
                    rounded_bw = round(exact_bw,3)
                elif metadata['bandwidth_gbps'] in [0.001,0.005]:
                    rounded_bw = round(exact_bw,4)
                # remaining cases are > 2.5G
                else: 
                    rounded_bw = round(2*exact_bw,0)/2
                group['bandwidth_gbps']     = rounded_bw
                # group['bandwidth_gbps']     = metadata['bandwidth_gbps']

            # Adjust the group info to the experiment type
            # .. that is, keep only the groups we want for that particular case
            if 'base' in dir.name:
                # .. keep only the experiment_type, i.e. 'base'
                group = {'exp_type': metadata['experiment_type']}
            elif 'idle' in dir.name:
                # .. remove port_speed and traffic data
                group.pop('port_speed', None)
                group.pop('packet_size_bytes', None)
                group.pop('bandwidth_gbps', None)
            else: 
                # .. remove traffic data
                if ('trx' in dir.name) or ('switch' in dir.name):
                    group.pop('packet_size_bytes', None)
                    group.pop('bandwidth_gbps', None)
                # .. fix the port speed units
                if old_speed:
                    if int(group['port_speed']) > 1000:
                        divider = 1000
                    else:
                        divider = 1
                    group['port_speed'] = str(int(group['port_speed']/divider))+'G'

            # Compute and store the corresponding power value
            power = analyse_exp(ts)
            store_datapoint(ts.name,power,power_data,group)

    # TODO: adjust sorting to get device, then base, then idle, ... 
    save_as_yml(power_data, output_data_path, 'power_data.yml', sort_keys=True)


In [ ]:
# Computing the model parameters from the power data
# Parsing all power data for a given device
# .. assumed ran from the `automation` directory

# Select a device to analyse
device_id = 'N540X-8Z16G-SYS-A'
# device_id = 'ciscoNexus9336-FX2'
# device_id = 'ciscoNCS55A1-24H'
# device_id = 'cisco8201-32FH'
# device_id = 'aristaDCS-7280CR3K-32D4'

# Set paths
input_data_path = Path('..','data','router',device_id,'static')
output_data_path = Path('..','devices',device_id)

power_model = {
    'metadata' : {
        'device_id': device_id,
        'connected_PSUs' : 2
    },
}


In [ ]:

    # Select a device to analyse
param = {
    'configuration': {
        # 'port_type':            'QSFP28',
        # 'port_type':            'QSFP',
        # 'port_type':            'RJ45',
        # 'port_type':            'SFP+',
        'port_type':            'SFP',
        # 'tranceiver_module' :   'LR',
        'tranceiver_module' :   'T',
        # 'tranceiver_module' :   'Passive DAC',
        # 'tranceiver_module' :   'PCC',
        # 'port_speed':           '100G',
        # 'port_speed':           '10G',
        'port_speed':           '1G',
        # 'port_speed':           '100M',
        # 'port_speed':           '10M',
    },
}
param.update(power_model)


power_model_per_config = derive_model(param)
power_model = push_data_in_dict(power_model,param['configuration'],power_model_per_config)
save_as_yml(power_model, output_data_path, 'power_model.yml', sort_keys=False)


In [ ]:

# param['configuration']['port_type'] = 'QSFP-DD'
# param['configuration']['port_speed'] = '400G'
# param['configuration']['tranceiver_module'] = 'FR4'

param['configuration']['port_speed'] = '50G'
power_model_per_config = derive_model(param)
power_model = push_data_in_dict(power_model,param['configuration'],power_model_per_config)

param['configuration']['port_speed'] = '25G'
power_model_per_config = derive_model(param)
power_model = push_data_in_dict(power_model,param['configuration'],power_model_per_config)


# print(yaml.dump(power_model, default_flow_style=False,sort_keys=False))
save_as_yml(power_model, output_data_path, 'power_model.yml', sort_keys=False)


## End of useful code

> Check the `PSU-data-validation` notebook instead

In [ ]:
# Validating the model with with the live data

# Set paths
traffic_data_path = Path('/nfs/nsg/group/jacobr/SWITCH_data/router-interfaces/zh3')
inventory_file = Path('/nfs/nsg/students/switch/switch-device-inventory/swizh3.txt')

total_power = pd.DataFrame(columns=['timestamp'])

interfaces = sorted(traffic_data_path.glob('**/*0_0_0_*.csv'))
# num_p = len(list(p))
# print(p)
counter = 0
for i in interfaces:

    # Filter out the vlan counters
    if len(i.stem.split('.')) != 1:
        continue
    print(i.name)
    df = pd.read_csv(i)

    # Increment coutner
    counter += 1

    # TODO: read the interface config from ... somewhere.
    i_config = 'config_a' 
    i_trx_power = power_model['P_INTERFACE'][i_config]['P_TRX']
    i_port_power = power_model['P_INTERFACE'][i_config]['P_PORT']
    # Read the interface power model parameters
    i_energy_per_bit = power_model['P_INTERFACE'][i_config]['E_BIT']
    i_energy_per_pkt = power_model['P_INTERFACE'][i_config]['E_PKT']

    # .. Column names are inconsistent. Use column indexes instead
    # df['total_bytes']   = df['ifHCInOctets']    + df['ifHCOutOctets']
    # df['total_pkt']     = df['ifHCInUcastPkts'] + df['ifHCOutUcastPkts']
    df['total_bytes']   = df.iloc[:,1] + df.iloc[:,2]
    df['total_pkt']     = df.iloc[:,5] + df.iloc[:,6]
    df['dyn_power']     = (i_energy_per_bit * df['total_bytes'] * 8 +
                           i_energy_per_pkt * df['total_pkt'])
    
    # Compute the total interface power
    tmp = pd.DataFrame()
    tmp['timestamp'] = df['timestamp']
    tmp[i.stem] = df['dyn_power'] + i_trx_power + i_port_power

    # Save the info the the global dataframe
    total_power = pd.merge(total_power, tmp, how='outer', on='timestamp')

    # if counter > 3:
    #     break

# Add the remaining power terms
p_base      = power_model['P_BASE']
p_offset    = power_model['P_INTERFACE'][i_config]['P_OFFSET']
total_power['p_base'] = p_base
total_power['p_offset'] = p_offset
# Set a proper datetime index
total_power.set_index('timestamp', inplace=True)
total_power.index = pd.to_datetime(total_power.index, unit='s')
# Sum it all up
total_power['p_total']= total_power.sum(axis=1)
display(total_power)


In [ ]:
# Filter the annoying outliers

start_time = '2024-10-01 00:00:00'
q_low = total_power["p_total"].quantile(0.01)
q_hi  = total_power["p_total"].quantile(0.99)
total_power = total_power[start_time:]

total_power_filtered = total_power[(total_power["p_total"] < q_hi) & (total_power["p_total"] > q_low)]

# Plot the modeled power
fig = px.scatter(total_power_filtered,x=total_power_filtered.index,y='p_total')
fig.show()

In [ ]:
# Compare with the PSU data

traffic_data_path = Path('/nfs/nsg/group/jacobr/SWITCH_data/router-power')
router_name = 'swizh3'
psu_data = pd.DataFrame()
for psu_id in [0,1]:
    file = traffic_data_path / (router_name + '_psu'+str(psu_id)+'.csv')
    tmp = pd.read_csv(file, index_col='timestamp')
    if 'mW' in tmp.columns[0]:
        tmp = tmp.div(1000)

    tmp.columns = [file.stem]
    # display(tmp)
    psu_data = tmp.join(psu_data)
        

# Compute the total
psu_data['swizh3_total'] = psu_data['swizh3_psu0'] + psu_data['swizh3_psu1']
psu_data.index = pd.to_datetime(psu_data.index,unit='s',utc=True)
display(psu_data)

psu_data = psu_data[start_time:]
fig.add_traces(
    list(px.scatter(psu_data,x=psu_data.index,y='swizh3_total').select_traces())
)
fig.show()



In [ ]:
# Compare with the autopower data

autopower_data_path = Path('~')
resampling = '30min'

autopower_data = pd.DataFrame()
for client_id in [17,18]:
    file = autopower_data_path / ('autopower'+str(client_id)+'.csv')


    tmp = pd.read_csv(file, index_col='measurement_timestamp')
    tmp.index = pd.to_datetime(tmp.index,format='mixed')
    
    tmp = tmp.resample(resampling).mean().div(1000)
    tmp.columns = ['autopower'+str(client_id)]
    # display(tmp)
    autopower_data = tmp.join(autopower_data)
        


In [ ]:

# Compute the total
autopower_data = autopower_data[start_time:]
autopower_data['autopower_total'] = autopower_data.iloc[:,0] + autopower_data.iloc[:,1]
display(autopower_data)

fig.add_traces(
    list(px.scatter(autopower_data,x=autopower_data.index,y='autopower_total').select_traces())
)
fig.show()



In [ ]:
display(autopower_data)

In [ ]:

interfaces = {
    'HundredGigE0/0/0/1' : {
        'PID'       : 'QSFP-100G-SR4-S',
        'format'    : 'QSFP28',
        'type'      : 'SR4',
        'phy'       : 'optical',
        'power(W)'  : 1
    }
}

